In [ ]:
!pip install python-dotenv
!pip install openai
!pip install langchain
!pip install pypdf
!pip install tiktoken
!pip install chromadb
!pip install lark
!pip install docarray
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 13.0 MB/s eta 0:00:00


## FOR GOOGLE COLLAB USERS ONLY: Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path='/content/drive/My Drive/14-Teaching/JoSSIS/txts/'

In [ ]:
vectorstore_path='/content/drive/My Drive/14-Teaching/JoSSIS/docs/'

## OpenAI Key

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-'

In [ ]:
import os
import openai
from dotenv import load_dotenv

try:
    # Check if OPENAI_API_KEY is already set and has 51 characters
    if 'OPENAI_API_KEY' in os.environ:
        if len(os.environ['OPENAI_API_KEY']) == 51:
            openai.api_key = os.environ['OPENAI_API_KEY']
            print("Key successfully acquired.")
        else:
            print(f"Key length is wrong, only {len(os.environ['OPENAI_API_KEY'])} characters")
            print("Searching for \"mykey\" file")
            # Load environment variables from "mykey" file
            load_status = load_dotenv("mykey", override=True)

            # Check if the .env file was successfully loaded
            if not load_status:
                raise EnvironmentError("Could not load the \"mykey\" file")

            # Retrieve the API key from environment variables
            openai.api_key = os.environ['OPENAI_API_KEY']

            # Check if the loaded API key has 51 characters
            if len(openai.api_key) != 51:
                raise ValueError("The API key must contain 51 characters")
            print("Key successfully acquired.")
    else:
        print("OPENAI_API_KEY is not set in environment variables.")

except KeyError:
    print("Error: OPENAI_API_KEY not found in environment variables.")

except EnvironmentError as env_err:
    print(f"Environment Error: {env_err}")

except ValueError as val_err:
    print(f"Value Error: {val_err}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

Key length is wrong, only 3 characters
Searching for "mykey" file
Key successfully acquired.


# Initalisation

Recall the overall workflow for retrieval augmented generation (RAG):

In [ ]:
llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


In [ ]:
persist_directory = 'docs/chroma/'
#persist_directory = vectorstore_path+'chroma/'
print(persist_directory)

docs/chroma/


# Document Loader


**Todo**: Set up the directory where you'll store your business strategy articles. Then, use LangChain's DirectoryLoader to load the documents from that directory.

# Document Splitting

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [ ]:
splits = text_splitter.split_documents(docs)

In [ ]:
len(splits)

2568

# Embeddings


In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [ ]:
print(persist_directory)

docs/chroma/


In [ ]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [ ]:
print(vectordb._collection.count())

2568


In [ ]:
vectordb.persist()

# Question Answering

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Beantworten Sie die Frage am Ende des Textes anhand \
der folgenden Informationen. Wenn Sie die Antwort nicht wissen, sagen Sie einfach, \
dass Sie es nicht wissen, versuchen Sie nicht, eine Antwort zu erfinden. \
Verwenden Sie maximal drei Sätze. Fassen Sie die Antwort so kurz wie möglich. \
Sagen Sie immer "Danke für die Frage" am Ende der Antwort.
{context}
Frage: {question}
Hilfreiche Antwort:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

**Todo**: Answer the following Questions with the help of RetrievalAQ



*  What is a Business Ecosystem?
* What are the core tasks of a Business Ecosystem Strategy?
* Do Business Ecosystems always include digital platforms?
* Why are Business Ecosystems attractive for companies today?
* How do Business Ecosystems require a new mindset compared to traditional business strategies?

